In [1]:
import pandas as pd
import carla
import math
import random
import time
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
client.set_timeout(200.0)
world  = client.get_world()
world = client.load_world('Town05_Opt')
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', "1280")
camera_bp.set_attribute('image_size_y', "720")
camera_bp.set_attribute('fov', "79.45")
camera_pose = carla.Transform(carla.Location(x=199.5, y=-90, z=8.5), carla.Rotation(pitch=-27.4, yaw=90, roll=0))
camera = world.spawn_actor(camera_bp, camera_pose)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# Get the map spawn points
spawn_points = world.get_map().get_spawn_points()

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [2]:
tm = client.get_trafficmanager(8000)

In [3]:
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

In [4]:
def project(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]
        
        point_camera /= point_camera[2]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)

        return point_img[0:2]

In [5]:
# Use camera intrinsic/extrinsic to have the exact same configuration than the real one

# Get the world to camera matrix
world_2_camera = np.array(camera_pose.get_inverse_matrix())
print(world_2_camera)

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

K

[[-3.88076451e-08  8.87815416e-01 -4.60199773e-01  8.38150940e+01]
 [-1.00000000e+00 -4.37113883e-08 -0.00000000e+00  1.99500000e+02]
 [-2.01159711e-08  4.60199773e-01  8.87815416e-01  3.38715515e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


array([[770.19972518,   0.        , 640.        ],
       [  0.        , 770.19972518, 360.        ],
       [  0.        ,   0.        ,   1.        ]])

In [6]:
# Set up the set of bounding boxes from the level
# We filter for traffic lights and traffic signs
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

# Remember the edge pairs
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]


In [7]:
# Unfortunately, there is a bug with current Carla version, and some vehicles, especially motorbikes,
# do not have correct bbox dimensions
my_vehicles = []
uncorrect_objects = ['vehicle.bh.crossbike', 'vehicle.diamondback.century', 'vehicle.harley-davidson.low_rider',
                     'vehicle.gazelle.omafiets', 'vehicle.kawasaki.ninja', 'vehicle.yamaha.yzf', 'vehicle.vespa.zx125']

for vehicle_bp in [bp for bp in bp_lib.filter('vehicle')]:
    if vehicle_bp.id not in uncorrect_objects:
        my_vehicles.append(vehicle_bp)

In [8]:
for spawn_point in spawn_points:
    vehicle_bp = random.choice(my_vehicles)
    npc = world.try_spawn_actor(vehicle_bp, spawn_point)
      
    if npc:
        npc.set_autopilot(True)
        tm.auto_lane_change(npc,False)
    else:
        print("Could not spwan actor")

In [9]:
idx = 1

while True:
    # Retrieve and reshape the image
    world.tick()
    image = image_queue.get()

    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

    # Get the camera matrix 
    world_2_camera = np.array(camera_pose.get_inverse_matrix())
    
    labels = []
    
    for npc in world.get_actors().filter('*vehicle*'):

        bb = npc.bounding_box
        dist = npc.get_transform().location.distance(camera_pose.location)
        
        # Get rid of box not directly on this highway
        x_pos_world = npc.get_transform().location.x
        if not 200 - 20 < x_pos_world < 200+20:
            continue

        # Filter for the vehicles within 50m
        if dist > 100:
            continue

        # Calculate the dot product between the forward vector
        # of the vehicle and the vector between the vehicle
        # and the other vehicle. We threshold this dot product
        # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
        forward_vec = camera_pose.get_forward_vector()
        ray = npc.get_transform().location - camera_pose.location

        if forward_vec.dot(ray) < 1:
            continue

        p1 = project(npc.get_transform().location, K, world_2_camera)
        #print(npc)

        # make sure we are not out of bounds
        if not (0 <= p1[0] < 1280 and 0 <= p1[1] < 720):
            continue

        verts = [v for v in bb.get_world_vertices(npc.get_transform())]
        for edge in edges:
            p1 = project(verts[edge[0]], K, world_2_camera)
            p2 = project(verts[edge[1]],  K, world_2_camera)

            #cv2.line(img, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), (255,0,0, 255), 1)


        middle = carla.Location(x=(verts[0].x+verts[6].x)/2, y = (verts[0].y+verts[6].y)/2, z=(verts[0].z+verts[6].z)/2)
        point_on_floor = project(middle,  K, world_2_camera)

        labels.append([point_on_floor[0], point_on_floor[1], bb.extent.y*2, bb.extent.z*2, bb.extent.x*2])
        #cv2.circle(img, (int(point_on_floor[0]), int(point_on_floor[1])), radius=3, color=(0, 255, 0), thickness=3)

    if len(labels) == 0:
        continue
    
    # Uncomment and change path to make your dataset
    #cv2.imwrite(f'/home/antoine/Documents/highway/carla_dataset_2/images/image_{idx:04d}.png', img)
    df = pd.DataFrame(np.array(labels), columns = ['x','y','w', 'h', 'l'])
    #df.to_csv(f'/home/antoine/Documents/highway/carla_dataset_2/labels/image_{idx:04d}.txt', index=False)
    cv2.imshow('ImageWindowName',img)
    if cv2.waitKey(1) == ord('q'):
        break
        
    idx += 1
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [10]:
np.set_printoptions(suppress=True)
world_2_camera

array([[ -0.00000004,   0.88781542,  -0.46019977,  83.81509399],
       [ -1.        ,  -0.00000004,  -0.        , 199.5       ],
       [ -0.00000002,   0.46019977,   0.88781542,  33.87155151],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [11]:
K

array([[770.19972518,   0.        , 640.        ],
       [  0.        , 770.19972518, 360.        ],
       [  0.        ,   0.        ,   1.        ]])